### Imports

In [22]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [23]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [24]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [25]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [26]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [27]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-89    4.9   47     0.43   0        Ruim     1.00      
-87    8.3   74     0.44   0        Ruim     1.00      
-82    6.9   98     0.34   0        Ruim     1.00      
-70    9.6   76     0.93   0        Ruim     1.00      
-62    8.7   61     0.82   0        Ruim     1.00      
-73    8.9   36     0.91   0        Ruim     1.00      
-80    4.7   62     0.74   0        Ruim     1.00      
-60    6.6   100    0.47   0        Ruim     1.00      
-77    9.9   78     0.43   0        Ruim     1.00      
-89    9.5   78     0.86   0        Ruim     1.00      
-95    8.3   98     0.57   0        Ruim     1.00      
-72    9.5   68     0.52   0        Ruim     1.00      
-90    7.5   48     0.90   0        Ruim     1.00      
-88    5.1   76     0.97   0        Ruim     1.00      
-85    8.9   79     0.87   0        Ruim     1.00      
-78    4.3   81     0.33   0        Ruim     1.0

### Save

In [28]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
